#  最終課題

課題のGithubリポジトリURLを提出してください＊Githubリポジトリは必ずpublicとしてください

武蔵野大学Webサイトのトップページにアクセス

同一ドメインの全てのリンク（コメントアウトされていないもの）を辿り，全ページのURLと<title>を辞書型変数に格納する

key：URL

value：<title></title>で挟まれた文字列

辞書型変数を print() で表示する


In [13]:
import requests
from bs4 import BeautifulSoup
import time

In [14]:
# URL クラス
class URL:
    def __init__(self, url, path = '', domain = 'musashino-u.ac.jp'):
        self.url = url
        self.path = path
        self.domain = domain

        self.Protocol = self.url.split(':')[0]
        if self.Protocol != 'http' and self.Protocol != 'https':
            raise ValueError("Unsupported protocol")
        split_url = self.url.split('/')

        if len(split_url) < 3:
            raise ValueError("Invalid URL format")    
        self.FQDN = self.url.split('/')[2]

        if len(split_url) < 4:
             self.Path = ''
        else:
            self.Path = '/'.join(self.url.split('/')[3:])


    # getProrocol https or http
    def get_Protocol(self):
        return self.Protocol
    
    # getFQDN
    def get_FQDN(self):
        return self.FQDN

    # getPath
    def get_Path(self):
        return self.Path
    
    # getAbsURL
    def get_abs_URL(self):
        #空の場合自身のURLを返す
        if not self.path:
            return self.url
        #絶対パスに変換
        protocol = self.path.split(':')[0]
        if protocol == 'http' or protocol == 'https':
            return self.path
        elif self.path[0] == '/':
            return f"{self.Protocol}://{self.FQDN}{self.path}"
        else:
            return f"{self.url}/{self.path}"
        
    # checkDomain  
    def check_domain(self):
        return self.domain in self.FQDN
    

In [15]:
# URLクラス動作確認
url = URL("http://もオン/index.html", "/example/path.html")
print(f"プロトコル: {url.get_Protocol()}")
print(f"FQDN: {url.get_FQDN()}")
print(f"パス: {url.get_Path()}")
print(f"URL: {url.get_abs_URL()}")
print(f"ドメインチェック: {url.check_domain()}")

プロトコル: http
FQDN: もオン
パス: index.html
URL: http://もオン/example/path.html
ドメインチェック: False


In [16]:
# PAGE クラス
class PAGE:
    def __init__(self, url, target_filetype = 'text/html'):
        self.url = url
        self.target_filetype = target_filetype
        self.valid = False
        self.links = []

        time.sleep(0.3)
        try:
            res = requests.get(self.url)
        except requests.exceptions.RequestException as e:
            self.pagestate = f"リクエスト失敗: {e}"
        res.encoding = res.apparent_encoding

        status_code = res.status_code
        headers = res.headers.get('Content-Type', '拡張子が見つかりませんでした。')

        if status_code != 200:
            self.pagestate = f"[{status_code}エラー] :"
        elif 'text/html' not in headers:
            self.pagestate =  f"[非HTML] : {headers}"        
        else:
            self.pagestate = "正常"
            self.valid = True
            self.soup = BeautifulSoup(res.text, 'html.parser')
            self.title = self.soup.find('title').string if self.soup.find('title') else 'タイトルなし'
            new_hrefs = self.soup.find_all(attrs={'href':True})
            for new_href in new_hrefs:
                new_url = new_href.get('href')
                target_url = URL(self.url, new_url)
                self.links.append(target_url.get_abs_URL())
                         
    #isValid
    def __private_is_Valid(self):
        if self.valid == False:
            raise ValueError("Invalid page")

    def is_Valid(self):
        return self.valid
    # requestStatus
    def check_Page(self):
        return self.pagestate
    
    # getPage
    def get_Soup(self):
        self.__private_is_Valid()
        return self.soup
    
    # getPageTitle
    def get_Title(self):
        self.__private_is_Valid()
        return self.title
    
    # getPageLinks
    def get_Links(self):
        self.__private_is_Valid()
        return self.links
        

In [17]:
# PAGEクラス動作確認
soup = PAGE("https://www.musashino-u.ac.jp/")
print(f"リクエスト状況: {soup.check_Page()}")
print(f"ページタイトル: {soup.get_Title()}")
print(f"リンク一覧: {soup.get_Links()}")

リクエスト状況: 正常
ページタイトル: 武蔵野大学
リンク一覧: ['https://www.musashino-u.ac.jp/', 'https://www.musashino-u.ac.jp/', 'https://www.musashino-u.ac.jp/en/', 'https://www.musashino-u.ac.jp/assets/img/common/favicon.ico', 'https://www.musashino-u.ac.jp/assets/img/common/apple-touch-icon_180x180.png', 'https://www.musashino-u.ac.jp/assets/img/common/android-chrome_256x256.png', 'https://fonts.googleapis.com', 'https://fonts.gstatic.com', 'https://fonts.googleapis.com/css2?family=BIZ+UDPGothic:wght@400;700&family=Cormorant+Garamond:wght@700&family=Hind:wght@700&display=swap', 'https://www.musashino-u.ac.jp/assets/css/destyle.min.css', 'https://www.musashino-u.ac.jp/assets/css/swiper-bundle.min.css', 'https://www.musashino-u.ac.jp/assets/css/scroll-hint.css', 'https://www.musashino-u.ac.jp/assets/ss/ss_search.css', 'https://www.musashino-u.ac.jp/assets/ss/ss_suggest.css', 'https://www.musashino-u.ac.jp/assets/css/main.css?20251110091222', 'https://www.musashino-u.ac.jp/assets/css/modules.css?20251110091222'

In [ ]:
#　メインコード
def scraping(start_url):
    to_scrap = [start_url]
    scraped = []
    page_dict = {}
    while to_scrap:

        #ページにアクセス
        current_url = to_scrap.pop(0)
        page_obj = PAGE(current_url)
        print(f" {page_obj.check_Page()}({len(scraped)}/{len(scraped) + len(to_scrap)}): {current_url}")

        # エラーページはスキップ
        if page_obj.is_Valid() == False:
            scraped.append(current_url)
            continue

        # 正常ページはリンクを取得
        else:
            title = page_obj.get_Title()
            urls = page_obj.get_Links()
            for url in urls:
                url_obj = URL(current_url, url)
                abs_url = url_obj.get_abs_URL()
                abs_url_obj = URL(abs_url)
                if (abs_url_obj.check_domain() and 
                    abs_url not in scraped and
                    abs_url not in to_scrap):
                    to_scrap.append(abs_url)
            scraped.append(current_url)
            page_dict[current_url] = title
        
    return page_dict

In [19]:
START_URL = "https://www.musashino-u.ac.jp/news/"

print(f"開始します: {START_URL}")

page_data = scraping(START_URL)

print(f"終了しました:{len(page_data)} ページを取得")

print("--- 辞書型変数の内容 ---")
for pages_data in page_data.items():
    print(pages_data)


開始します: https://www.musashino-u.ac.jp/news/
 正常(0/0): https://www.musashino-u.ac.jp/news/
 正常(1/124): https://www.musashino-u.ac.jp/news/index.php
 [500エラー] :(2/132): https://www.musashino-u.ac.jp/en/news/
 [非HTML] : image/vnd.microsoft.icon(3/132): https://www.musashino-u.ac.jp/assets/img/common/favicon.ico
 [非HTML] : image/png(4/132): https://www.musashino-u.ac.jp/assets/img/common/apple-touch-icon_180x180.png
 [非HTML] : image/png(5/132): https://www.musashino-u.ac.jp/assets/img/common/android-chrome_256x256.png
 [非HTML] : text/css(6/132): https://www.musashino-u.ac.jp/assets/css/destyle.min.css
 [非HTML] : text/css(7/132): https://www.musashino-u.ac.jp/assets/css/swiper-bundle.min.css
 [非HTML] : text/css(8/132): https://www.musashino-u.ac.jp/assets/css/scroll-hint.css
 [非HTML] : text/css(9/132): https://www.musashino-u.ac.jp/assets/ss/ss_search.css
 [非HTML] : text/css(10/132): https://www.musashino-u.ac.jp/assets/ss/ss_suggest.css
 [非HTML] : text/css(11/132): https://www.musashino-u.a

KeyboardInterrupt: 

# 　拡張したいコード

エラーが発生したときにスクレイピングが途中から復旧できるような構造にしたい

In [ ]:
import csv

In [ ]:
# [TODO: DATAクラスの実装]
# DATA クラス
class DATA:
    def __init__(self, url, title, state = 'new'):
        self.url = url
        self.title = title
        self.state = state

        self.data.append({
            'url': self.url,
            'state': self.state,
            'title': self.title
        })
        


    # saveData　データを保存するURLリストが追加されるたびに保存
    def save_data(self):
        #csvに入れます
        with open('assignment.csv', 'w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerows(self.data)
        
    # roadData　データを読み込む
    def load_data(self):
        #csvから読み込みます
        try:
            with open('assignment.csv', 'r', newline='', encoding='utf-8') as f:
                # 1. csv.readerオブジェクトを作成
                reader = csv.reader(f)
                
                # 2. readerをリストに変換して、全データを一括で読み込む
                # (readerはイテレータなので、forループで1行ずつ処理することも可能です)
                data_list = list(reader)

            # 読み込んだデータを表示して確認
            print(f"--- {f'assignment.csv'} から読み込んだデータ ---")
            for row in data_list:
                print(row)
                # print(f"読み込んだデータ全体: {data_list}") # こちらでもOK

        except FileNotFoundError:
            print(f"エラー: ファイル 'assignment.csv' が見つかりません。")
        except Exception as e:
            print(f"エラーが発生しました: {e}")
        

    # checkData　URLリストのデータと新しく追加したデータを照合
    def check_data(self):
     

SyntaxError: incomplete input (2263637427.py, line 51)